In [5]:
import shapely.geometry
import pyproj

In [1]:
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [11]:
# Set up projections
p_ll = pyproj.Proj(init='epsg:4326')
p_mt = pyproj.Proj(init='epsg:3857') # metric; same as EPSG:900913

# Create corners of rectangle to be transformed to a grid
nw = shapely.geometry.Point((-5.0, 40.0))
se = shapely.geometry.Point((-4.0, 41.0))

stepsize = 5000 # 5 km grid step size

# Project corners to target projection
s = pyproj.transform(p_ll, p_mt, nw.x, nw.y) # Transform NW point to 3857
e = pyproj.transform(p_ll, p_mt, se.x, se.y) # .. same for SE

# Iterate over 2D area
gridpoints = []
x = s[0]
while x < e[0]:
    y = s[1]
    while y < e[1]:
        p = shapely.geometry.Point(pyproj.transform(p_mt, p_ll, x, y))
        gridpoints.append(p)
        y += stepsize
    x += stepsize

TypeError: a bytes-like object is required, not 'str'

In [17]:
import shapely.geometry
import pyproj

from utils import midpoint
from entrie import Entrie, EntrieType 
from cell import Cell

class Grid:
    def __init__(self, cells):
        self.cells = cells

    def add_entrie(self, entrie: Entrie, type: EntrieType):
        c = entrie.coorditates
        for i in range(len(self.cells) - 1):
            m = midpoint(self.cells[i][0].center, self.cells[i + 1][0].center)
            if c[0] > m[0]:
                for j in range(len(self.cells[i]) - 1):
                    m = midpoint(self.cells[i][j].center, self.cells[i][j + 1].center)
                    if m[1] > c[1]:
                        self.cells[i][j].add_entrie(entrie, type)
                        return i, j

    def make_grid(top_left, bottom_right, initial_size=100):
        """Factory method for creating Grid instances"""
        
        # Set up projections
        p_ll = pyproj.Proj(init='epsg:4326')
        p_mt = pyproj.Proj(init='epsg:3857') # metric; same as EPSG:900913

        # Create corners of rectangle to be transformed to a grid
        nw = shapely.geometry.Point(top_left)
        se = shapely.geometry.Point(bottom_right)

        stepsize = initial_size

        # Project corners to target projection
        s = pyproj.transform(p_ll, p_mt, nw.x, nw.y) # Transform NW point to 3857
        e = pyproj.transform(p_ll, p_mt, se.x, se.y) # .. same for SE

        # Iterate over 2D area
        cells = []
        x = s[0]
        while x > e[0]:
            y = s[1]
            row = []
            while y < e[1]:
                nw_point = shapely.geometry.Point(pyproj.transform(p_mt, p_ll, x, y))
                se_point = shapely.geometry.Point(pyproj.transform(p_mt, p_ll, x + stepsize, y + stepsize))
                row.append(Cell(midpoint((nw_point.x, nw_point.y), (se_point.x, se_point.y))))
                y += stepsize
            cells.append(row)
            x -= stepsize

        return Grid(cells)

In [18]:
nw = (60.140862, 29.932493)
se = (59.738452, 30.654168)

grid = Grid.make_grid(nw, se, 5000)

In [19]:
len(grid.cells)

9

In [20]:
i, j = grid.add_entrie(Entrie((59.84890953266116, 30.068633938742236)), EntrieType.VACANCY)

(59.84890953266116, 30.068633938742234) (60.14086199999999, 29.951951505593648)
(59.84890953266116, 30.068633938742234) (60.09594623579401, 29.951951505593648)
(59.84890953266116, 30.068633938742234) (60.051030471588035, 29.951951505593648)
(59.84890953266116, 30.068633938742234) (60.00611470738206, 29.951951505593648)
(59.84890953266116, 30.068633938742234) (59.961198943176086, 29.951951505593648)
(59.84890953266116, 30.068633938742234) (59.91628317897012, 29.951951505593648)
(59.84890953266116, 30.068633938742234) (59.87136741476414, 29.951951505593648)
(59.84890953266116, 30.068633938742234) (59.82645165055816, 29.951951505593648)


In [21]:
i, j

(7, 3)

In [22]:
grid.cells[7][3].entries

{<EntrieType.VACANCY: 1>: [<entrie.Entrie at 0x7f78f409b668>],
 <EntrieType.APARTMENT: 2>: [],
 <EntrieType.SHOP: 3>: [],
 <EntrieType.BUS_STOP: 4>: [],
 <EntrieType.TRAM_STOP: 5>: [],
 <EntrieType.SUBWAY_STATION: 6>: [],
 <EntrieType.CAFE: 7>: [],
 <EntrieType.OFFICE: 8>: [],
 <EntrieType.POINT_OF_INTEREST: 9>: []}

In [25]:
midpoint(grid.cells[0][0].center, grid.cells[0][1].center)

(60.16331988210298, 29.97140620171343)

In [26]:
midpoint(grid.cells[0][2].center, grid.cells[0][3].center)

(60.16331988210298, 30.04919449246976)

In [6]:
from utils import midpoint

In [7]:
for i in range(len(grid.cells) - 1):
    m = midpoint(grid.cells[0][i].center, grid.cells[i + 1][0].center)
    if m[0] > c[0]:
        for j in range(len(grid.cells[i]) - 1):
            if c[1] > m[1]:
                return 

(60.14086199999999, 29.951951505593648)

In [ ]:
if m[0] < c[0]